# 0.) Importing necessary packages

In [2]:
from flask import Flask
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

# 1.) Read in CSV file with all return values

In [12]:
returns = pd.read_csv('returns.csv')
returns.drop('Unnamed: 0', axis=1, inplace=True)
returns

,Fund,1yr,2yr,3yr,7yr,10yr
0,fundOIEIX,-1.381160,2.387870,13.965800,9.51689,9.74620
1,fundJLGRX,-0.273622,-5.528020,13.373300,17.27030,15.43520
2,fundJGASX,-3.220000,-6.962650,13.188900,15.64010,15.41170
3,fundFBGRX,-2.847750,-9.802180,13.703600,16.00340,15.42590
4,fundPEMPX,-2.740400,-7.565150,0.412597,1.95308,1.18070
5,fundGSCGX,1.827580,-0.958647,13.804900,12.67770,12.31420
6,fundGGOIX,-1.157030,-10.960600,8.801320,10.41230,9.93794
7,fundVEIPX,0.179058,4.590800,14.507700,9.83061,10.08030
8,fundGPIRX,-1.338140,-3.203880,6.810770,5.33730,4.60770
9,fundVIVIX,-0.507799,3.665700,14.941200,10.06130,10.31860


# 2.) Creating UI Dashboard function

In [13]:
def create_dashboard():
    returns_trial = returns.set_index('Fund')
    returns_transposed = returns_trial.T
    df1 = returns_transposed.reset_index()
    df1 = df1.rename(columns={'index': 'period'}).rename_axis(None, axis=1)
    
    tickers_ = [df1.columns[1:][i][4:] for i in range(len(df1.columns[1:]))]   #for checkbox display
    
    # Incorporate data
    funds = df1.columns[1:]
    options = [{"label": tickers_[i], "value": funds[i]} for i, _ in enumerate(funds)]

    # Initialize the app
    app = Dash(__name__)

    # App layout
    app.layout = html.Div([
        html.H1('Mutual Funds - Annualized Returns'),
        html.Hr(),
        dcc.Checklist(options=options,inline=True, value=['fundOIEIX'],id='checklist'),
        html.Br(),
        dash_table.DataTable(data=df1.to_dict('records')),
        dcc.Graph(figure={}, id='controls-and-graph')           
    ])

    # Add controls to build the interaction
    @callback(
        Output(component_id='controls-and-graph', component_property='figure'),
        Input(component_id='checklist', component_property='value')
    )
    def update_graph(col_chosen):
        colors = px.colors.sequential.Blues
        fig = px.histogram(df1, x='period', y=col_chosen, histfunc='avg',
                           color_discrete_sequence=colors[8:3:-1])
        fig.update_layout(
        barmode='group',
        bargroupgap=0.2,
        xaxis=dict(title='Period'),
        yaxis=dict(title='Annualized Returns'),
        legend_title_text='Funds'
    )
        return fig

    # Run the app
    if __name__ == '__main__':
        app.run_server()

# 3.) Running UI

Click on the link generated and explore the data!

In [16]:
create_dashboard()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
